In [ ]:
%matplotlib inline

# misc. libraries
import pandas as pd
import numpy as np

# plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# ml libraries
from sklearn import cluster, metrics
from sklearn.decomposition import PCA

import inspect

%load_ext autoreload
%autoreload 2

In [ ]:
# local dependencies
from load import *
from helpers import *
from plots import *
from constants import *

**General notes**
* The aim is to find out if the tumor is responding to a specific hormone. This response is "induced" in mice by treating them with the hormone. In humans we try to find similar expression patterns to determine if the tumor is driven by a certain hormone. In which case we can group such tumors together for more targeted and better treatment. We know that the patients have a certain type of cancer and this is recorded somewhere although we don't have this information now. If we discover a clear clustering, then it will be valuable to see which cancer types each data point (patient) has. If we interpret each cluster as being certain type of cancer instead of a hormone response, then we will misinterpret the results since the genetic expressions (features) are not results
* [Patient derived xenograft, Wikipedia](https://en.wikipedia.org/wiki/Patient_derived_xenograft)
* [Few useful things to know about ML](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)
* Useful magics:
  * `%pycat <filename>` to show a syntax-highlighted file
  * `%psource <object>` prints the source code for an object (function, for instance)

**Notes from Fabio**
* The data include: a matrix from our PDX models stimulated with different hormones (estrogen, progesterone and testosterone) - from which I estimated a list of differentially expressed genes to interrogate the patients' datasets - and a matrix from breast cancer patients retrieved from the TCGA (the Cancer Genome Atlas Consortium - published data).
* I suggest you to start testing the list of differentially expressed genes on your training data (PDXs, which is labelled with the correct treatment) that you can use as positive control in order to test the performance of your methods (you can estimate the sensitivity and specificity of the methods that you want to use, so that we can have an idea of which method should be in principle better to use in the patients dataset). Once you will be able to correctly discriminate the samples in the training set, then you can start interrogating the patients' matrix.
* Keep in mind that one of the early things that we do when dealing with sequencing data is to get rid of genes that show no or "minor" expression overall in the dataset. This means that, for those genes that do not reach a given threshold of expression in a number of patients, they are simply excluded from the analysis because they are supposedly not informative and just confounding the further analysis. Therefore, to some degree, we should expect to have differences in the genes that are expressed. 
* Regarding the patients data I already gave you they should be already normalized, whereas the new patients' dataset that you downloaded from the Internet  you should make sure it is correctly normalized (for this you should more or less have a normal distribution of your counts and more or less the samples - columns- should have a similar number of counts) and then run the analysis. However, I the interpretation of the PCA plot will not be trivial: since these are patients samples they are likely to be heterogeneous and therefore I do not expect to have a clear clustering of the samples based on their cancer type.
* [Informations about drugs used to treat breast cancer](https://www.nature.com/articles/d41573-019-00201-w) over the last decades, to give you an idea of the need for a more personalized medicine.
* [Small review on personalized medicine and why it should be pursued](https://notendur.hi.is/~vol1/pdx-papers/nejmsb1503104.pdf)
* [Patient-derived xenograft models of breast
cancer and their predictive power](https://notendur.hi.is/~vol1/pdx-papers/2015%20PDX%20Breast%20Cancer%20Research.pdf)
* [A paper from our lab in which there is the description of our PDX (Patient-Derived Xenograft) model - your training dataset](https://notendur.hi.is/~vol1/pdx-papers/2016%20MIND%20for%20Breast%20Cancer%20Sflomos%20Cancer%20Cell.pdf)

+ **DHT:** Dihydrotestosterone is an endogenous androgen sex steroid and hormone
+ **E2:** Estradiol (E2), also spelled oestradiol, is an estrogen steroid hormone and the major female sex hormone
+ **P4:**  Progesterone (P4) is an endogenous steroid and progestogen sex hormone involved in the menstrual cycle, pregnancy, and embryogenesis of humans and other species

***
**Data loading and manipulation**
***

In [ ]:
%psource load_genes

In [ ]:
# Raw information about genes
genes = load_genes()
genes

In [ ]:
# Preprocessed list of genes
genes_list = load_genes_list()
genes_list.head(2)

In [ ]:
# Genes showing response to two hormones
genes_list[genes_list[["dht", "e2", "p4"]].sum(axis=1) == 2]

In [ ]:
# Load TCGA first tumor-patients dataset
patients = load_patients()

In [ ]:
# Load TCGA second tumor-patients dataset
patients2 = load_patients2()

Not all the genes are expressed in every cell of patients and within a broad set of samples, so we discard genes not expressed in the patients datasets. This is what is to be expected, especially when taking into account that we are comparing data on human patients with data from a xenograft experiment. Let's list the genes that were not found:

In [ ]:
genes_not_found = genes_list.genes[~genes_list.genes.isin(patients.columns)]
print(f"Genes not found in the patients datasets:\n\n{list(genes_not_found)}")

In [ ]:
genes_expressed = patients.columns
print(f"Genes found in the patients datasets:\n\n{list(genes_expressed)}")

Let's now load the patient derived xenograft (PDX) experiment data:

In [ ]:
pdx = load_pdx()
pdx

***
**Exploratory data analysis**
***

Let's investigate how correlated features are among each other in the first patients dataset. We shall plot a heatmap to visualize the lower triangular Pearson correlation matrix.

In [ ]:
%psource df_to_tril

In [ ]:
patients_corr = df_to_tril(patients.corr())

In [ ]:
# %pycat plots

In [ ]:
plot_corr(patients_corr, genes_expressed, "corr_patients")

We observe that most of the genes are rather uncorrelated, which gives us a hint of their linear independence.

Now let's look at the pairs of genes which are highly correlated. We define our threshold to be 0.6.

In [ ]:
patients_genes_corr = patients_corr[patients_corr > CORR_THRESHOLD].stack()

In [ ]:
patients_genes_corr.index = patients_genes_corr.index.tolist()
patients_genes_corr.name = "patients_correlation"

In [ ]:
series = []

for h in HORMONES:
    # get all genes expressing hormone h
    genes_h = genes_list[genes_list[h]]
    glist = list(genes_h.genes)

    num_genes = len(glist)

    # Compute pairs of indices for the lower triangular part of a matrix
    # of size (num_genes x num_genes), excluding the diagonal
    # (we don't want to pair the genes to themselves)
    tril_indices = np.tril_indices(num_genes, k=-1)
    index_pairs = list(zip(tril_indices[0], tril_indices[1]))

    # We map the list of index-pairs to all possible pairs of genes
    pairs = [(glist[pair[0]], glist[pair[1]]) for pair in index_pairs]

    # idx = pd.MultiIndex.from_tuples(genes_pairs)
    series_h = pd.Series(data=h, index=pairs, name="pdx_hormone")
    series.append(series_h)
    
genes_pairs = pd.concat(series, sort=False).groupby(level=0).apply(list)

In [ ]:
genes_matches = genes_pairs[genes_pairs.index.isin(patients_genes_corr.index)]

In [ ]:
patients_genes_corr.sort_values(ascending=False, inplace=True)
patients_genes_corr

In [ ]:
(
    pd.DataFrame(patients_genes_corr)
    .join(genes_matches)
)

There are appear to be even more correlations in the second patients dataset that match the PDX data.
Can we therefore conclude that we can expect better/consistent results for this dataset 
when we run the methods trained on the PDX data?

In [ ]:
patients2_corr = patients2.corr()
patients2_corr = df_to_tril(patients2_corr)

In [ ]:
patients2_genes_corr = patients2_corr[patients2_corr > CORR_THRESHOLD].stack()

In [ ]:
patients2_genes_corr.index = patients2_genes_corr.index.tolist()
patients2_genes_corr.name = "patients2_correlation"

In [ ]:
genes_matches = genes_pairs[genes_pairs.index.isin(patients2_genes_corr.index)]

In [ ]:
patients2_genes_corr.sort_values(ascending=False, inplace=True)
patients2_genes_corr

In [ ]:
(
    pd.DataFrame(patients2_genes_corr)
    .join(genes_matches)
)

***
Let's now analyze the distribution of the features (genetic expressions). Do we observe anything abnormal about the distributions? Will we detect outliers? How will we handle them?

In [ ]:
len(genes_expressed)

In [ ]:
patients.iloc[:, 0:10]

In [ ]:
plot_feature_distributions(patients, genes_expressed)

In [ ]:
# todo: analyze distributions, decide what to do with outliers

***
**Feature processing**
***

Let's try to reduce the dimensionality of the input space, i.e. the linear mapping of our D-dimensional input into a K-dimensional space K<=D that best represents the original data.

In [ ]:
# INCOMPLETE
# PCA decomposition of original gene list
# we want to verify that the pre-selected genes are linearly independent

pca = PCA()
pca.fit(pdx)
PCA(copy=True, iterated_power='auto', svd_solver='auto', tol=0.0, whiten=False)

y_pos = np.arange(len(pca.singular_values_))
plt.bar(y_pos, pca.singular_values_, align='center', alpha=0.5)
plt.ylabel('Values')
plt.xlabel('Principal components')
plt.title('PCA - Singular values')
plt.show()

y_pos = np.arange(len(pca.singular_values_))
plt.bar(y_pos, pca.explained_variance_, align='center', alpha=0.5)
plt.ylabel('Explained variance')
plt.xlabel('Principal components')
plt.title('PCA - explained variance')
plt.show()

In [ ]:
# plot data explained by 2nd and 3rd principal component
pca.n_components = 3
X_reduced = pca.fit_transform(pdx)
X_reduced = np.append(X_reduced, pdx.label.values.reshape((33,1)), axis=1)
plt.plot(X_reduced[0:3,1], X_reduced[0:3,2], 'ro')
plt.plot(X_reduced[3:14,1], X_reduced[3:14,2], 'bo')
plt.plot(X_reduced[14:23,1], X_reduced[14:23,2], 'co')
plt.legend(['dht', 'p4', 'e2'])
#plt.plot(X_reduced[23:,1], X_reduced[24:,2], 'yo')
#plt.legend(['dht', 'p4', 'e2', 'ctrl'])
plt.xlabel('2nd PC')
plt.ylabel('3rd PC')
plt.show()

In [ ]:
# interactive 3D plot of first 3 principal components

# uncomment below line to have interactive plot!
# %matplotlib notebook

pca.n_components = 3
X_reduced = pca.fit_transform(pdx)
labels = pdx.label.values.reshape((33,1))
X_reduced = np.append(X_reduced, labels, axis=1)
fig = plt.figure()
ax = plt.axes(projection='3d')
Axes3D.scatter(ax, X_reduced[:3,0], X_reduced[:3,1], X_reduced[:3,2])
Axes3D.scatter(ax, X_reduced[3:14,0], X_reduced[3:14,1], X_reduced[3:14,2])
Axes3D.scatter(ax, X_reduced[14:23,0], X_reduced[14:23,1], X_reduced[14:23,2])
#Axes3D.scatter(ax, X_reduced[23:,0], X_reduced[23:,1], X_reduced[23:,2])
ax.set_xlabel('1st PC')
ax.set_ylabel('2nd PC')
ax.set_zlabel('3rd PC')
ax.legend(['dht', 'p4', 'e2'])
#ax.legend(['dht', 'p4', 'e2', 'ctrl'])

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)

pca = PCA()
pca.fit(pdx)

fig, ax = plt.subplots()
xi = np.arange(0, 33, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.3,1.1)
plt.plot(xi, y, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 33, step=1)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.99, color='r', linestyle='-')
plt.axhline(y=0.95, color='orange', linestyle='-')

plt.text(0.5, 1, '99% cut-off threshold', color = 'red', fontsize=10)
plt.text(0.5, 0.9, '95% cut-off threshold', color = 'orange', fontsize=10)

ax.grid(axis='x')
plt.show()

print(pca.n_components_)

***
**Clustering**
***

In [ ]:
X = pdx.drop("label", axis=1)
pdx_labeled = pdx.label

clus = cluster.AgglomerativeClustering(n_clusters=4)  # , affinity='manhattan', linkage='average')
predicted = clus.fit_predict(X)

# calculate score
score = metrics.adjusted_rand_score(pdx_labeled, predicted)
print(score)
# accuracy, f2 = performance(predicted)

***
**Spectral Clustering**
***

In [ ]:
clustering = cluster.SpectralClustering(assign_labels="discretize", random_state=0).fit(X)
print("predicted labels : " + str(clustering.labels_))
print("true labels :      " + str(pdx_labeled.values))
print("Score : " + str(metrics.adjusted_rand_score(pdx_labeled, clustering.labels_)))

***
**K-Means**
***

In [ ]:
kmeans = cluster.KMeans(n_clusters=4, random_state=0).fit(X)
print("predicted labels : " + str(kmeans.labels_))
print("true labels :      " + str(pdx_labeled.values))
print("Score : " + str(metrics.adjusted_rand_score(pdx_labeled, kmeans.labels_)))

In [ ]:
# we should rather evaluate with the metrics.adjusted_rand_score function 

def performance(labels):
    '''Evaluate performance of predicted cluster compared to pre-selected gene list'''
    # get gene list
    geneNP = genes_list.loc[:,'dht':'p4'].astype(int).values  # replace with Boolean values
    
    nb_clusters = len(np.unique(labels))
    accuracy = np.zeros([nb_clusters, 3])
    f2 =  np.zeros([nb_clusters, 3])
    beta = 2
    for i in np.arange(nb_clusters):
        label = np.zeros_like(labels)
        label[labels == i] = 1
        for j in np.arange(geneNP.shape[1]):
            # plot confusion matrices
            
            #cm = metrics.confusion_matrix(geneNP[:,j], label)
            #cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            #fig, ax = plt.subplots()
            #im = ax.imshow(cm, interpolation='nearest')
            #ax.figure.colorbar(im, ax=ax)
            accuracy[i,j] = np.mean(geneNP[:,j]==label)
            f2[i,j] = metrics.fbeta_score(geneNP[:,j], label, beta)
    return accuracy, f2
        

***
**Principal Component Analysis**
***